In [145]:
import pandas as pd
import random

</p>
0 - русский
1 - белорусский
2 - украинский
3 - болгарский
4 - македонский
5 - сербский
<p>

In [147]:
languages_dict = {'rus' : 0, 'bel' : 1, 'ukr' : 2, 'bul' : 3, 'mac' : 4, 'srp' : 5}
text_sample_size = 140
def id2lang(id):
    for i, key in languages_dict.items():
        if key == id:
            return i

In [138]:
def define_alphabet():
    slav = 'абвгдежзклмнопрстуфхцчш'
    rus  = 'ёийщъыьэюя'
    bel  = 'ёійўыьэюя'
    uk   = 'ґєиіїйщьюя'
    bul  = 'ийщъьюя'
    mac  = 'ѓѕијљњќџ'
    srp  = 'ђијљњћџ'
    special_chars = ' ,!?'
    
    all_lang_chars = slav + rus + bel + uk + bul + mac + srp
    small_chars = list(set(list(all_lang_chars)))
    small_chars.sort() 
    big_chars = list(set(list(all_lang_chars.upper())))
    big_chars.sort()
    small_chars += special_chars
    letters_string = ''
    letters = small_chars + big_chars
    for letter in letters:
        letters_string += letter
    return small_chars,big_chars,letters_string

In [141]:
alphabet = define_alphabet()
print(alphabet)

(['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'ђ', 'ѓ', 'є', 'ѕ', 'і', 'ї', 'ј', 'љ', 'њ', 'ћ', 'ќ', 'ў', 'џ', 'ґ', ' ', ',', '!', '?'], ['Ё', 'Ђ', 'Ѓ', 'Є', 'Ѕ', 'І', 'Ї', 'Ј', 'Љ', 'Њ', 'Ћ', 'Ќ', 'Ў', 'Џ', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'Ґ'], 'абвгдежзийклмнопрстуфхцчшщъыьэюяёђѓєѕіїјљњћќўџґ ,!?ЁЂЃЄЅІЇЈЉЊЋЌЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯҐ')


In [150]:
# count number of chars in text based on given alphabet
def count_chars(text,alphabet):
    alphabet_counts = []
    for letter in alphabet:
        count = text.count(letter)
        alphabet_counts.append(count)
    return alphabet_counts

# process text and return sample input row for DNN
# note that we are counting separatey:
# a) counts of all letters regardless of their size (whole text turned to lowercase letter)
# b) counts of big letters only
# this is because German uses big letters for beginning of nouns so this feature is meaningful
def get_input_row(content):
    counted_chars_all = count_chars(sample_text.lower(),alphabet[0])
    counted_chars_big = count_chars(sample_text,alphabet[1])
    all_parts = counted_chars_all + counted_chars_big
    return all_parts

In [152]:
sample_text = "Ну шо посоны, ониме"
print ("1. Sample text: \n",sample_text)
print ("2. Reference alphabet: \n",alphabet[0],alphabet[1])
sample_input_row = get_input_row(sample_text)
print ("3. Sample_input_row: \n",sample_input_row)
input_size = len(sample_text)
print ("4. Input size : ", input_size)

1. Sample text: 
 Ну шо посоны, ониме
2. Reference alphabet: 
 ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'ђ', 'ѓ', 'є', 'ѕ', 'і', 'ї', 'ј', 'љ', 'њ', 'ћ', 'ќ', 'ў', 'џ', 'ґ', ' ', ',', '!', '?'] ['Ё', 'Ђ', 'Ѓ', 'Є', 'Ѕ', 'І', 'Ї', 'Ј', 'Љ', 'Њ', 'Ћ', 'Ќ', 'Ў', 'Џ', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'Ґ']
3. Sample_input_row: 
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 3, 4, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4. Input size :  19


In [67]:
X_val = pd.read_csv('./task2-dev.csv')

In [136]:
X_val

,language,text
0,0,Олег Шилов самостоятельно сделал мачту
1,0,Олег Шилов учился на факультете
2,0,Из-за пропуска военных лагерей был
3,0,Олег Шилов стал работать со
4,0,На европейских чемпионатах Хопёрский был
5,0,Впоследствии многие тренеры спрашивали у
6,0,Эта победа стала первой для
7,0,Экспресс-сообщение по линии не осуществляется
8,0,Между этой станцией и следующей
9,0,Виной всему стало неудовлетворительное техниче...
